<a href="https://colab.research.google.com/github/JulienVm/NLP_Project/blob/main/NLP_Project_Julien_VAN_MIEGEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projet NLP - Analyse de sentiments



## Importations

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from nltk.tokenize import word_tokenize
from nltk.text import Text
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer
import nltk

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

## Création du set de données

In [ ]:
nRowsRead = 10000 # le nombre de lignes qu'on lit dans notre csv (cela prendrait trop de temps de tout lire)
offset=0 # pour décaler les indexes de début et de fin de nos lignes lues dans le csv
nRowsSkipped = [i for i in range(offset+nRowsRead-nRowsRead//2,1599998-nRowsRead//2-offset)] # liste des indexes des lignes non lues dans notre csv
data=pd.read_csv("drive/MyDrive/NLP/data.csv", encoding="ISO-8859-1", skiprows=nRowsSkipped, nrows=nRowsRead, header=None, names=['target','id','date','flag','user','text'], dtype={'target':np.int32,'id':np.int32,'date':np.string_,'flag':np.string_,'user':np.string_,"text":np.string_})
data.head() # on a un dataframe à 6 colonnes : le sentiment entre 0 pour négatif, 2 pour neutre et 4 pour positif, l'id, la date, le flag, l'utilisateur et le texte

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


##Extraction des données utiles

In [ ]:
# selon la consigne de l'exercice, on doit retirer les tweets neutres labélisés à 2
indexToDelete=[] # liste des indexes des lignes à enlever dans notre dataframe
for i in range(len(data['target'])):
  if data['target'][i]==2:
    indexToDelete.append(i)
print("Removed",len(indexToDelete),"rows.")
data.drop(indexToDelete) # drop permet d'enlever les lignes correspondant aux indexes listés

Removed 0 rows.


,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
9995,4,-2101387819,Tue Jun 16 08:39:00 PDT 2009,NO_QUERY,ChloeAmisha,@SCOOBY_GRITBOYS
9996,4,-2101387807,Tue Jun 16 08:39:00 PDT 2009,NO_QUERY,EvolveTom,"@Cliff_Forster Yeah, that does work better tha..."
9997,4,-2101365330,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
9998,4,-2101365327,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...


##Normalisation

In [ ]:
feelingList=data['target']/4 # on normalise les labels de tweets positifs à 1 au lieu de 4, et on met le tout dans une liste
tweetList=data['text'] # liste contenant les tweets
tweetList[0:5]

0    @switchfoot http://twitpic.com/2y1zl - Awww, t...
1    is upset that he can't update his Facebook by ...
2    @Kenichan I dived many times for the ball. Man...
3      my whole body feels itchy and like its on fire 
4    @nationwideclass no, it's not behaving at all....
Name: text, dtype: object

## Mise en forme des données




###Formation de vecteurs de mots

In [ ]:
from nltk.stem.porter import *
stemmer=stemmer = PorterStemmer() # on prend le porter stemmer car c'est en anglais
from nltk.stem import WordNetLemmatizer
Word_Lemmatizer = WordNetLemmatizer()

for i in range(len(tweetList)):
  tweet=""
  for char in tweetList[i]:
    if char not in [',','!','(',')','?',':','[',']',';','-','+']: # on remplace la ponctuation par des points
      tweet+=char
    else:
      tweet+='.'
  tweet=tweet.lower() # on met en minuscule
  tweet=word_tokenize(tweet) # tokenisation
  tweet=[w for w in tweet if not w in list(nltk.corpus.stopwords.words('english'))] # on enlève les stopwords
  tweet=[stemmer.stem(w) for w in tweet] #stemming = on enlève la conjugaison, les pluriels ...
  tweet=[Word_Lemmatizer.lemmatize(w) for w in tweet] # lemmatisation = on ne garde que la racine des mots
  tweet=[w for w in tweet if ('@' not in w) and ('http' not in w)] # on enlève certains mots jugés inutiles (mentions, liens)
  tweetList[i]=tweet
tweetList[0:5]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


0    [switchfoot, ., awww, ., 's, bummer, ., should...
1    [upset, ca, n't, updat, facebook, text, ..., m...
2    [kenichan, dive, mani, time, ball, ., manag, s...
3               [whole, bodi, feel, itchi, like, fire]
4    [nationwideclass, ., 's, behav, ., 'm, mad, .,...
Name: text, dtype: object

###Création du set de vocabulaire

In [ ]:
V=set() # notre vocabulaire, un ensemble contenant chaque mot rencontré dans les tweets étudiés
for tweet in tweetList :
  for word in tweet :
    V.add(word)
len(V)

17182

###Création de bag of words

In [ ]:
list_bags=[] # liste qui contiendra les bags of words
for tweet in tweetList:
  bag=np.zeros(len(V)) # vecteur de 0 de dimension égale à la taille du vocabulaire
  i=0
  for word in V:
    if word in tweet :
      bag[i]=tweet.count(word) # à chaque fois qu'un mot du vocablaire est dans un tweet, on met à la coordonnée du vecteur correspondant à ce mot le nombre d'occurences du mot dans le tweet
    i+=1
  list_bags.append(bag)

list_bags=np.array(list_bags)
finalData=pd.DataFrame(list_bags,columns=V) # dataframe pour visualiser nos données finales
finalData.head()

,gtalk,becoz,otah,j,andrerib,spiral,myki,greed,quot.hous,elbow,soundcheck,rori,homestead,file,camilla,masih,class..,robertzalm,whoschrishugh,today.,tme,despr,boymonst,jacob,hold,writer/poet,quot.drag,gruel,wanted2,martin,auto.small.cap,button..goodnight,movingtomontana,jeremi,josordoni,rachael90210,aberdeen,var,catchbifucan,citrusandcandi,...,syifa,faerywitch,stole,workng,179,wz,ttsc,august,librari,die,v.upset,shiti,brng,hou,megmcc,missytigg,candychao,senior,nerdi,dang,slacker,brought,'goocharama,week..,inesvarga,.aka,puberty..,ceiling.,ill.,carpet,saffc,muscl,pad,pissed/hungri,expeci,gulf,marcofrissen,quot.which,kazzc22,musik
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Sauvegarde du prétraitement

In [ ]:
pickle_out = open("drive/MyDrive/NLP/list_bags.pickle","wb") # on sauvegarde list_bags (les bag of words) pour ne pas avoir à refaire le prétraitement lorsque l'on réexécute le notebook
pickle.dump(list_bags, pickle_out)
pickle_out.close()

pickle_out = open("drive/MyDrive/NLP/feelingList.pickle","wb") # idem pour feelingList (liste des labels)
pickle.dump(feelingList, pickle_out)
pickle_out.close()

pickle_out = open("drive/MyDrive/NLP/V.pickle","wb") # idem pour V (le vocabulaire)
pickle.dump(V, pickle_out)
pickle_out.close()

##Création du modèle simple de référence


In [ ]:
# on importe les données sauvegardées
# Lorsqu'on réexécute le notebook, il suffit d'exécuter la cellule au début pour les importations, puis on passe directement à celle-ci
pickle_in=open("drive/MyDrive/NLP/list_bags.pickle","rb")
x=pickle.load(pickle_in)
pickle_in.close()

pickle_in=open("drive/MyDrive/NLP/feelingList.pickle","rb")
y=pickle.load(pickle_in)
pickle_in.close()

pickle_in=open("drive/MyDrive/NLP/V.pickle","rb")
V=pickle.load(pickle_in)
pickle_in.close()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
import seaborn as sn
from sklearn import linear_model

from sklearn.linear_model import LogisticRegression

x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.8,random_state=0) # on sépare nos données en un jeu de test et un jeu d'entraînement

modele_regLog = linear_model.LogisticRegression(random_state = 0,solver = 'liblinear', multi_class = 'auto') # initialisation d'un modèle simple de régression logistique

modele_regLog.fit(x_train,y_train) # entraînement du modèle

precision = modele_regLog.score(x_test,y_test) # test du modèle
print("Nous avons une précision de",precision*100)

Nous avons une précision de 75.25


##Un premier RNN

### Création d'un premier RNN

In [ ]:
from keras.models import Sequential
from keras.models import load_model
from keras.layers.core import Dense
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Flatten
from os.path import exists

if exists("drive/MyDrive/NLP/model1.h5"):
  model1=load_model("drive/MyDrive/NLP/model1.h5",compile=False) # on charge le modèle s'il existe déjà
else :
  model1=Sequential() # sinon, on en itialise un nouveau
  model1.add(Dense(64, activation='relu')) # c'est un premier modèle "naïf" qui n'utilise qu'une couche dense
  model1.add(Dense(1, activation='sigmoid'))

### Entraînement d'un premier RNN

In [ ]:
model1.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model1.fit(x_train, y_train,epochs=5, batch_size=10, verbose=1) # entraînement du modèle

Epoch 1/5
800/800 [==============================] - 2s 3ms/step - loss: 0.0000e+00 - accuracy: 0.5092
Epoch 2/5
800/800 [==============================] - 2s 3ms/step - loss: 0.0000e+00 - accuracy: 0.5062
Epoch 3/5
800/800 [==============================] - 2s 3ms/step - loss: 0.0000e+00 - accuracy: 0.5056
Epoch 4/5
800/800 [==============================] - 2s 3ms/step - loss: 0.0000e+00 - accuracy: 0.5144
Epoch 5/5
800/800 [==============================] - 2s 2ms/step - loss: 0.0000e+00 - accuracy: 0.4954


###Sauvegarde

In [ ]:
model1.save("drive/MyDrive/NLP/model1.h5",overwrite=True) # on sauvegarde le drive dans le drive

###Test du RNN

In [ ]:
model1.evaluate(x_test,y_test) # on évalue le modèle sur les données de test

63/63 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 1.5683


[0.0, 1.568253993988037]

In [ ]:
def pretraitement(initTweet): # fonction qui reprend la méthode de prétraitement qu'on a vu plus haut : elle prend en entrée une phrase et on retourne le bag of words
  tweet=""
  for char in initTweet:
    if char not in [',','!','(',')','?',':','[',']',';','-','+']: # on remplace la ponctuation par des points
      tweet+=char
    else:
      tweet+='.'
  tweet=tweet.lower() # on met en minuscule
  tweet=word_tokenize(tweet) # tokenisation
  tweet=[w for w in tweet if not w in list(nltk.corpus.stopwords.words('english'))] # on enlève les stopwords
  tweet=[stemmer.stem(w) for w in tweet] #stemming = on enlève la conjugaison, les pluriels ...
  tweet=[Word_Lemmatizer.lemmatize(w) for w in tweet] # lemmatisation = on ne garde que la racine des mots
  tweet=[w for w in tweet if ('@' not in w) and ('http' not in w)] # on enlève certains mots jugés inutiles (mentions, liens...)
  bag=np.zeros(len(V))
  i=0
  for word in V:
    if word in tweet :
      bag[i]=tweet.count(word)
    i+=1
  return bag

In [ ]:
test="I hate" # testons notre pipeline avec une phrase
test=pretraitement(test)
print(model1.predict(np.array([test]))[0][0])

0.0


##On teste avec l'API

In [ ]:
import requests
import json
api_key='guzYPRiLoRC1YWkpo0UMfFofIR9WykNMEZmtCBhazcCyvkohbkirOlslWZ-siZr-3pXr7XDcdoxSpczkyEp0uMJr_148T3kHKmjTd99saCVON3e-_qRhEC3Q8EI6YHYx'
headers = {'Authorization': 'Bearer %s' % api_key}

url = "https://api.yelp.com/v3/businesses/FEVQpbOPOwAPNIgO7D3xxw/reviews" # n'ayant pas trouvé le json de tweets, je prends des commentaires de restaurants
req = requests.get(url, headers=headers)
print('the status code is {}'.format(req.status_code))


dataAPI=json.loads(req.text)
dataAPI

the status code is 200


{'possible_languages': ['fr', 'en', 'zh', 'pt', 'de', 'it', 'sv', 'ja', 'es'],
 'reviews': [{'id': 'qOiuCTz_Gzgmose1GHaAlg',
   'rating': 5,
   'text': "Truth be told if it was up to me I'd be giving 4/5 stars, we did order recently and our cheese fries weren't delivered, couldn't speak with anyone from...",
   'time_created': '2020-12-19 07:52:35',
   'url': 'https://www.yelp.com/biz/shake-shack-new-york-2?adjust_creative=j_T35sgsu3IpWm0V3qF5Zg&hrid=qOiuCTz_Gzgmose1GHaAlg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=j_T35sgsu3IpWm0V3qF5Zg',
   'user': {'id': 'FRKnLBnlFiasr1Dc8oOIGQ',
    'image_url': 'https://s3-media3.fl.yelpcdn.com/photo/1wm0yjWSw92j_ZsUFZBGzQ/o.jpg',
    'name': 'Sarah G.',
    'profile_url': 'https://www.yelp.com/user_details?userid=FRKnLBnlFiasr1Dc8oOIGQ'}},
  {'id': '7d56A_ObMPHHyywcfhnrUw',
   'rating': 5,
   'text': 'Happened to be in the city today and not too far from here so I had to stop by and pick up cheese fries (I love those c

In [ ]:
nTests=3

for i in range(nTests):
  testData =dataAPI["reviews"][i]["text"]
  print(testData)
  testData=pretraitement(testData)
  prediction=model1.predict(np.array([testData]))[0][0] # prediction de notre modèle entre 0 et 1
  if (prediction<0.4):
    print("This comment is negative.")
  elif (prediction>0.6):
    print("This comment is postive.")
  else:
    print("This comment is neutral.")

Truth be told if it was up to me I'd be giving 4/5 stars, we did order recently and our cheese fries weren't delivered, couldn't speak with anyone from...
This comment is negative.
Happened to be in the city today and not too far from here so I had to stop by and pick up cheese fries (I love those crinkle cuts!) and a shake. Yum. Tbh,...
This comment is negative.
Before I moved to NY, my first ever trip to the city was in 2005 - this was when Shake Shack had this one lone original location, and it was an event to go...
This comment is negative.


##Second RNN plus évolué

###Nouvelle forme de prétraitement avec Keras

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

nRowsRead = 20000 # le nombre de lignes qu'on lit dans notre csv
offset=0 # pour décaler les indexes de début et de fin de nos lignes lues dans le csv
nRowsSkipped = [i for i in range(offset+nRowsRead-nRowsRead//2,1599998-nRowsRead//2-offset)] # liste des indexes des lignes non lues dans notre csv
data2=pd.read_csv("drive/MyDrive/NLP/data.csv", encoding="ISO-8859-1", skiprows=nRowsSkipped, nrows=nRowsRead, header=None, names=['target','id','date','flag','user','text'], dtype={'target':np.int32,'id':np.int32,'date':np.string_,'flag':np.string_,'user':np.string_,"text":np.string_})

for i in range(len(data2["text"])):
    data2["text"][i]=data2["text"][i].lower()

nWords=5000 # nombre de mots de vocabulaire qu'on garde : on ne garde que les plus
tokenizer = Tokenizer(num_words=nWords, split=" ") # faisons un essai avec le prétraitement de keras : il sera peut-être plus adapté pour le RNN
tokenizer.fit_on_texts(data2["text"].values)

x2 = tokenizer.texts_to_sequences(data2["text"].values)
x2 = pad_sequences(x2)

y2=pd.get_dummies(data2["target"]).values # on met 0 1 pour 0 et 1 0 pour 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


###Création

In [ ]:
if exists("drive/MyDrive/NLP/model2.h5"):
  model2=load_model("drive/MyDrive/NLP/model2.h5",compile=False) # on charge le modèle s'il existe déjà
else :
  model2=Sequential() # sinon, on en itialise un nouveau
  model2.add(Embedding(nWords,128,input_length=x2.shape[1])) # l'Embedding layer nous permet de convertir chaque mot en un vecteur de longueur fixée, qui a des valeurs autres que 0 et 1
  model2.add(Dropout(0.3)) # pour éviter l'overfitting
  model2.add(LSTM(128,return_sequences=False,dropout=0.3,recurrent_dropout=0.2)) # le lstm permet de repérer l'information utile
  model2.add(Dense(32, activation='relu'))
  model2.add(Dense(2, activation='sigmoid'))

###Entraînement

In [ ]:
x2_train,x2_test,y2_train,y2_test = train_test_split(x2,y2,train_size=0.8,random_state=0) # on sépare nos données en un jeu de test et un jeu d'entraînement

In [ ]:
model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model2.fit(x2_train, y2_train,epochs=15, batch_size=20, verbose=1) # entraînement du modèle

Epoch 1/15
800/800 [==============================] - 51s 62ms/step - loss: 0.0669 - accuracy: 0.9724
Epoch 2/15
800/800 [==============================] - 49s 61ms/step - loss: 0.0617 - accuracy: 0.9757
Epoch 3/15
800/800 [==============================] - 48s 60ms/step - loss: 0.0560 - accuracy: 0.9790
Epoch 4/15
800/800 [==============================] - 49s 62ms/step - loss: 0.0515 - accuracy: 0.9788
Epoch 5/15
800/800 [==============================] - 49s 61ms/step - loss: 0.0474 - accuracy: 0.9811
Epoch 6/15
800/800 [==============================] - 48s 60ms/step - loss: 0.0449 - accuracy: 0.9816
Epoch 7/15
800/800 [==============================] - 50s 62ms/step - loss: 0.0432 - accuracy: 0.9821
Epoch 8/15
800/800 [==============================] - 49s 62ms/step - loss: 0.0408 - accuracy: 0.9827
Epoch 9/15
800/800 [==============================] - 51s 64ms/step - loss: 0.0367 - accuracy: 0.9859
Epoch 10/15
800/800 [==============================] - 49s 61ms/step - loss: 0.029

###Sauvegarde

In [ ]:
model2.save("drive/MyDrive/NLP/model2.h5",overwrite=True)  # on évalue le modèle sur les données de test

###Test

In [ ]:
model2.evaluate(x2_test,y2_test) # on évalue le modèle sur les données de test

125/125 [==============================] - 2s 12ms/step - loss: 1.9131 - accuracy: 0.7275


[1.91311776638031, 0.7275000214576721]

In [ ]:
nTests=3

for i in range(nTests):
  testData =dataAPI["reviews"][i]["text"]
  print(testData)
  testData=pretraitement(testData)
  prediction=model2.predict(np.array([testData]))[0][0] # prediction de notre modèle entre 0 et 1
  if (prediction<0.4):
    print("This comment is negative.")
  elif (prediction>0.6):
    print("This comment is postive.")
  else:
    print("This comment is neutral.")

Truth be told if it was up to me I'd be giving 4/5 stars, we did order recently and our cheese fries weren't delivered, couldn't speak with anyone from...
0.4752633
This comment is neutral.
Happened to be in the city today and not too far from here so I had to stop by and pick up cheese fries (I love those crinkle cuts!) and a shake. Yum. Tbh,...
0.4752633
This comment is neutral.
Before I moved to NY, my first ever trip to the city was in 2005 - this was when Shake Shack had this one lone original location, and it was an event to go...
0.4752633
This comment is neutral.


In [ ]:
prediction=model2.predict(np.array([pretraitement("I hate this")]))[0][0] # testons notre pipeline avec une phrase
print(prediction)
if (prediction<0.4):
  print("This comment is negative.")
elif (prediction>0.6):
  print("This comment is postive.")
else:
  print("This comment is neutral.")

prediction=model2.predict(np.array([pretraitement("I love this")]))[0][0] # testons notre pipeline avec une phrase
print(prediction)
if (prediction<0.4):
  print("This comment is negative.")
elif (prediction>0.6):
  print("This comment is postive.")
else:
  print("This comment is neutral.")

0.4752633
This comment is neutral.
0.4752633
This comment is neutral.
